[![Ask me on Discord](https://img.shields.io/badge/Ask%20me%20on%20Discord-blue?style=for-the-badge&logo=https://simpleicons.org/icons/discord.svg&link=https://lookup.guru/512044995696525322)](https://lookup.guru/512044995696525322)  [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=for-the-badge)](https://github.com/DEX-1101/sd-webui-notebook/blob/main/kaggle-colab-notebook.ipynb) [![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fgithub.com%2FDEX-1101%2Fsd-webui-notebook%2Fblob%2Fmain%2Fkaggle-colab-notebook.ipynb&countColor=%2337d67a)](https://visitorbadge.io/status?path=https%3A%2F%2Fgithub.com%2FDEX-1101%2Fsd-webui-notebook%2Fblob%2Fmain%2Fkaggle-colab-notebook.ipynb)

In [ ]:
import os

update_webui = True
update_extensions = True
branch = "master"
root_path = "/kaggle/working" # change to "/content" if using colab.
webui_path = os.path.join(root_path, "x1101")
config = "https://raw.githubusercontent.com/DEX-1101/sd-webui-notebook/main/res/config.json" # import your webui config here, if have one
#=================================================

def install_req():
    if root_path == "/kaggle/working":
        !pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
        !pip install -q xformers==0.0.20 triton==2.0.0 -U
    else:
        !pip install -q xformers==0.0.22.post7

print("Please wait...")
!pip install -q git+https://github.com/DEX-1101/colablib
import time
from colablib.colored_print import cprint, print_line
from colablib.utils import py_utils
from IPython.display import clear_output
start_time    = time.time()
clear_output()

%cd {root_path}
if not os.path.exists(webui_path):
    print_line(0, color="green")
    cprint("[+] Preparing Environment", color="flat_cyan")
    cprint("    Installing requirements...", color="flat_yellow")
    !apt-get install lz4 &> /dev/null
    !pip install -q colorama
    !apt -y install -qq aria2 &> /dev/null
    !npm install -g localtunnel &> /dev/null
    !curl -s -OL https://github.com/DEX-1101/sd-webui-notebook/raw/main/res/new_tunnel --output-dir {root_path}
    !curl -s -Lo /usr/bin/cl https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cl
    !wget -q https://raw.githubusercontent.com/gutris1/segsmaker/main/script/pantat88.py -O {root_path}/semvak_zeus.py
    !curl -sLO https://github.com/openziti/zrok/releases/download/v0.4.23/zrok_0.4.23_linux_amd64.tar.gz --output-dir {root_path}
    !tar -xzf zrok_0.4.23_linux_amd64.tar.gz && rm -rf zrok_0.4.23_linux_amd64.tar.gz && mv {root_path}/zrok /usr/bin
    
    cprint("    Installing dependency...", color="flat_yellow")
    install_req()
    cprint("    Done!", color="flat_yellow")
    print_line(0, color="green")
    
    cprint("[+] Downloading Repository...", color="flat_cyan")
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M https://huggingface.co/x1101/UI/resolve/main/ui.tar.lz4 -o ui.tar.lz4
    cprint("    Unpacking...", color="flat_yellow")
    !tar -xI lz4 -f ui.tar.lz4
    !mv {root_path}/kaggle/working/x1101 {root_path}
    !rm {root_path}/ui.tar.lz4
    !rm -rf {root_path}/kaggle
    !wget -q {config} -O {webui_path}/config.json
    !wget -q https://raw.githubusercontent.com/DEX-1101/SecretNAI/main/template.txt -O {root_path}/download_list.txt
        
    # Custom code here ================
    !git clone -q https://github.com/Zyin055/Config-Presets {webui_path}/extensions/config-presets
    !wget -q https://github.com/DEX-1101/sd-webui-notebook/raw/main/res/config-txt2img.json -O {webui_path}/extensions/config-presets/config-txt2img.json
    # =================================
else:
    print_line(0, color="green")
    cprint("[+] Repository already exists. Skipping...", color="flat_cyan")

if update_webui:
    print_line(0, color="green")
    cprint("[+] Updating Repository...", color="flat_cyan")
    %cd {webui_path}
    !git pull &> /dev/null
    !git switch {branch}
    !git pull
    !git reset --hard

if update_extensions:
    print_line(0, color="green")
    cprint("[+] Updating Extensions...", color="flat_cyan")
    %cd {webui_path}/extensions
    !find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} reset --hard \; &> /dev/null
    !find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} pull \; &> /dev/null
    !find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} reset --hard \;
    
elapsed_time  = py_utils.calculate_elapsed_time(start_time)
print_line(0, color="green")
cprint(f"[+] All completed within {elapsed_time}.", color="flat_cyan")

### Download Manual

In [ ]:
civitai_api = "" # Put your api key here if error downloading from civitai

import os
oppai = f"sed -i 's|toket = \"?token=YOUR_API_KEY\"|toket = \"?token={civitai_api}\"|g' {root_path}/semvak_zeus.py"
os.system(oppai)
%run {root_path}/semvak_zeus.py
#================================================

# Models
%cd {webui_path}/models/Stable-diffusion
%download https://civitai.com/api/download/models/275408

# Lora
%cd {webui_path}/models/Lora
%download https://huggingface.co/x1101/sdxl-model/resolve/main/acheron/acheron/acheron.safetensors

# Embeddings   
%cd {webui_path}/embeddings
%download https://civitai.com/api/download/models/245812?token=(your_api_key_here)

# ControlNet    
%cd {webui_path}/extensions/sd-webui-controlnet/models/
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml

### Download from Pastebin

In [ ]:
pastebin_url = "https://pastebin.com/mr0v09pG"
hf_token = "" # if use private repo HuggingFace
#=================================================

#Code by Bang Furqanil
import os
import time
from colablib.utils import py_utils
from pydantic import BaseModel
from colablib.utils.py_utils import get_filename
from colablib.sd_models.downloader import aria2_download, download
from colablib.colored_print import cprint, print_line
from colablib.utils.config_utils import read_config
from colablib.utils.git_utils import clone_repo

custom_model_url        = ""
custom_vae_url          = ""
custom_embedding_url    = ""
custom_LoRA_url         = ""
custom_extensions_url   = ""
models_dir          = os.path.join(webui_path, "models", "Stable-diffusion")
vaes_dir            = os.path.join(webui_path, "models", "VAE")
lora_dir            = os.path.join(webui_path, "models", "Lora")
embeddings_dir      = os.path.join(webui_path, "embeddings")
extensions_dir      = os.path.join(webui_path, "extensions")
download_list       = os.path.join(root_path, "download_list.txt")
class CustomDirs(BaseModel):
    url: str
    dst: str
user_header = f"Authorization: Bearer {hf_token}"
custom_dirs = {
    "model"       : CustomDirs(url=custom_model_url, dst=models_dir),
    "vae"         : CustomDirs(url=custom_vae_url, dst=vaes_dir),
    "embedding"   : CustomDirs(url=custom_embedding_url, dst=embeddings_dir),
    "lora"        : CustomDirs(url=custom_LoRA_url, dst=lora_dir),
    "extensions"  : CustomDirs(url=custom_extensions_url, dst=extensions_dir),
}

def parse_urls(filename):
    content = read_config(filename)
    lines   = content.strip().split('\n')
    result  = {}
    key     = ''
    for line in lines:
        if not line.strip():
            continue
        if line.startswith('//'):
            continue
        if line.startswith('#'):
            key = line[1:].lower()
            result[key] = []
        else:
            urls = [url.strip() for url in line.split(',') if url.strip() != '']
            result[key].extend(urls)
    return result

def get_filename(url, token=None):
    headers = {}
    if token:
        headers['Authorization'] = f'Bearer {hf_token}'
       
def custom_download(custom_dirs):
    for key, value in custom_dirs.items():
        urls     = value.url.split(",")  # Split the comma-separated URLs
        dst      = value.dst

        if value.url:
            print_line(0, color="green")
            cprint(f" [+] Downloading {key}.", color="flat_yellow")

        for url in urls:
            url = url.strip()  # Remove leading/trailing whitespaces from each URL
            if url != "":
                print_line(0, color="green")
                if "|" in url:
                    url, filename = map(str.strip, url.split("|"))
                    if not filename.endswith((".safetensors", ".ckpt", ".pt", "pth")):
                        filename = filename + os.path.splitext(get_filename(url))[1]
                else:
                    if not url.startswith("fuse:"):
                        filename = get_filename(url)

                if url.startswith("fuse:"):
                    fuse(url, key, dst)
                elif key == "extensions":
                    clone_repo(url, cwd=dst)
                else:
                   download(url=url, filename=filename, user_header=user_header, dst=dst, quiet=False)

def download_from_textfile(filename):
    for key, urls in parse_urls(filename).items():
        for url in urls:
            if "civitai.com" in url:
                url += "&ApiKey={civitai_api_key}" if "?" in url else "?ApiKey={civitai_api_key}"
        key_lower = key.lower()
        if key_lower in custom_dirs:
            if custom_dirs[key_lower].url:
                custom_dirs[key_lower].url += ',' + ','.join(urls)
            else:
                custom_dirs[key_lower].url = ','.join(urls)
        else:
            cprint(f"Warning: Category '{key}' from the file is not found in custom_dirs.", color="flat_yellow")
            
def custom_download_list(url):
    filename = "custom_download_list.txt"
    filepath = os.path.join(root_path, filename)
    if os.path.exists(filepath):
        os.remove(filepath)
    if 'pastebin.com' in url:
        if 'raw' not in url:
            url = url.replace('pastebin.com', 'pastebin.com/raw')
    download(url=url, filename=filename, user_header=user_header, dst=root_path, quiet=True)
    return filepath

def main():
    start_time    = time.time()
    textfile_path = download_list
    if pastebin_url:
        textfile_path = custom_download_list(pastebin_url)
    download_from_textfile(textfile_path)
    custom_download(custom_dirs)

    elapsed_time  = py_utils.calculate_elapsed_time(start_time)
    print_line(0, color="green")
    cprint(f"[+] Download completed within {elapsed_time}.", color="flat_yellow")
main()

### Start WebUI

In [ ]:
ngrok_token = ""
zrok_token = "" # optional, good ngrok alternative
password = "x1101"

#====================================================================

import re
import requests
from colorama import init, Fore, Back, Style
from colablib.colored_print import cprint, print_line
import time
try:
    start_colab
except:
    start_colab = int(time.time())-5
    
def get_public_ip(version='ipv4'):
    try:
        url = f'https://api64.ipify.org?format=json&{version}=true'
        response = requests.get(url)
        data = response.json()
        public_ip = data['ip']
        
        return public_ip
    except Exception as e:
        print(f"Error getting public {version} address:", e)
        
public_ipv4 = get_public_ip(version='ipv4')

%cd {root_path}
import cloudpickle as pickle
tunnel_class = pickle.load(open("new_tunnel", "rb"), encoding="utf-8")
tunnel_port= 1101
tunnel = tunnel_class(tunnel_port)
tunnel.add_tunnel(command="cl tunnel --url localhost:{port}", name="cl", pattern=re.compile(r"[\w-]+\.trycloudflare\.com"))
tunnel.add_tunnel(command="lt --port {port}", name="lt", pattern=re.compile(r"[\w-]+\.loca\.lt"), note="Password : " + Fore.GREEN + public_ipv4 + Style.RESET_ALL + " rerun cell if 404 error.")
if zrok_token:
    !zrok enable {zrok_token} &> /dev/null
    tunnel.add_tunnel(command="zrok share public http://localhost:{port}/ --headless", name="zrok", pattern=re.compile(r"[\w-]+\.share\.zrok\.io"))

with tunnel:
    #!python -m http.server 1101
    %cd {webui_path}
    !echo -n {start_colab} >{webui_path}/static/colabTimer.txt
    !sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {webui_path}/modules/shared_options.py
    !python launch.py --port=1101 --ngrok {ngrok_token}  --encrypt-pass={password} --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --disable-safe-unpickle --no-half-vae

### HuggingFace Upload

In [ ]:
token = ""

# Make Repo
repo_name = "models"
private_repo = True 

# Upload
model_repo = "x1101/models"
model_path = "/kaggle/working/x1101/models/Stable-diffusion/0.5(AAMGape) + 0.5(AnythingV5Ink_v5PrtRE).safetensors"
path_in_repo = ""
commit_message = ""

#==============================================================
%pip install huggingface_hub &> /dev/null
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
login(token, add_to_git_credential=True)

api = HfApi()
user = api.whoami(token)

if repo_name:
    try:
        model_repo = user["name"] + "/" + repo_name.strip()
        validate_repo_id(model_repo)
        api.create_repo(repo_id=model_repo, private=private_repo)
        print("Repository doesn't exists, creating Repository...")
        print("Repository: ", model_repo, "created!\n")

    except HfHubHTTPError as e:
        print(f"Repository : {model_repo} exists, skipping create Repository\n")

if not commit_message:
  commit_message = "feat: upload"

def upload_model(model_paths, is_folder :bool, is_config :bool):
  path_obj = Path(model_paths)
  trained_model = path_obj.parts[-1]

  if path_in_repo:
        trained_model = path_in_repo

  if is_folder:
    print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
    print(f"Please wait...")

    api.upload_folder(
        folder_path=model_paths,
        path_in_repo=trained_model,
        repo_id=model_repo,
        commit_message=commit_message,
        ignore_patterns=".ipynb_checkpoints"
        )
    print(f"Upload success, located at https://huggingface.co/"+model_repo+"/tree/main\n")
  else:
    print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
    print(f"Please wait...")

    api.upload_file(
        path_or_fileobj=model_paths,
        path_in_repo=trained_model,
        repo_id=model_repo,
        commit_message=commit_message,
        )

    print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")

def upload():
    if model_path.endswith((".ckpt", ".safetensors", ".pt")):
      upload_model(model_path, False, False)
    else:
      upload_model(model_path, True, False)

upload()